### Clean and condense Portland 2024 CVRs for city council STV election.
### This notebook: District 1

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True

from tqdm import tqdm

In [2]:
D1_df = pd.read_csv("./Original CVRs/City_of_Portland__Councilor__District_1_2024_11_29_17_26_12.cvr.csv")

In [3]:
D1_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON","Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON","Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Timur Ender:NON","Choice_21_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Timur Ender:NON","Choice_22_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Noah Ernst:NON","Choice_22_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Noah Ernst:NON","Choice_23_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Candace Avalos:NON","Choice_23_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Candace Avalos:NON","Choice_24_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Joe Allen:NON","Choice_24_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Joe Allen:NON","Choice_25_1:City of Portland, Councilor, District 1:1:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:2:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:3:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:4:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:5:Number of Winners 3:David Linn:NON","Choice_25_1:City of Portland, Councilor, District 1:6:Number of Winners 3:David Linn:NON","Choice_26_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Cayle Tern:NON","Choice_26_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Cayle Tern:NON","Choice_27_1

### Note that this is all the voters in the city, not just in D1.

In [4]:
list(D1_df.columns)

['RowNumber',
 'BoxID',
 'BoxPosition',
 'BallotID',
 'PrecinctID',
 'BallotStyleID',
 'PrecinctStyleName',
 'ScanComputerName',
 'Status',
 'Remade',
 'Choice_20_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_20_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Peggy Sue Owens:NON',
 'Choice_21_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Timur Ender:NON',
 'Choice_21_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Timur Ender:NON',
 'Choice_21_1:City of Portland, Councilor, District 1:3:Number of Winners

### Here are all the D1 candidates that the CVR contains rankings for:

In [5]:
D1_candidates = [
    'Candace Avalos',
    'Jamie Dunphy',
    'Loretta Smith',
    'Noah Ernst',
    'Terrence Hayes',
    'Steph Routh',
    'Timur Ender',
    'Doug Clove',
    'Peggy Sue Owens',
    'David Linn',
    'Joe Allen',
    'Michael (Mike) Sands',
    'Deian Salazar',
    'Cayle Tern',
    'Thomas Shervey',
    'Joe Furi',
    'Uncertified Write In',
    'Write-in-120',
    'Write-in-121',
    'Write-in-122'
]

### Collect columns where 1st place, etc. votes are tallied so we can look for overvotes/undervotes while converting to a better format.

In [6]:
rank1_cols = [col for col in D1_df.columns if '1:Number' in col]
rank2_cols = [col for col in D1_df.columns if '2:Number' in col]
rank3_cols = [col for col in D1_df.columns if '3:Number' in col]
rank4_cols = [col for col in D1_df.columns if '4:Number' in col]
rank5_cols = [col for col in D1_df.columns if '5:Number' in col]
rank6_cols = [col for col in D1_df.columns if '6:Number' in col]

all_rank_cols = rank1_cols + rank2_cols + rank3_cols + rank4_cols + rank5_cols + rank6_cols

### This dataset has rows for ALL Portland voters; trim it down to voters who cast at least one vote.

In [7]:
D1_voters_df = D1_df[D1_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True)

### Add a column for each raw ranking:

In [8]:
D1_voters_df.insert(10, "Rank1 raw", None)
D1_voters_df.insert(11, "Rank2 raw", None)
D1_voters_df.insert(12, "Rank3 raw", None)
D1_voters_df.insert(13, "Rank4 raw", None)
D1_voters_df.insert(14, "Rank5 raw", None)
D1_voters_df.insert(15, "Rank6 raw", None)


### For each voter and each rank, record whether the voter overvoted or undervoted in that rank, or if they voted for exactly one candidate in that rank, record that candidate.

In [9]:
for i in tqdm(D1_voters_df.index):
    
    rank1_votes = D1_voters_df.loc[i, rank1_cols].sum()
    if rank1_votes == 0:
        D1_voters_df.loc[i, 'Rank1 raw'] = "skipped"
    elif rank1_votes > 1:
        D1_voters_df.loc[i, 'Rank1 raw'] = "overvote"
    elif rank1_votes == 1:
        this_vote_col = [col for col in rank1_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank1 raw'] = this_vote

    rank2_votes = D1_voters_df.loc[i, rank2_cols].sum()
    if rank2_votes == 0:
        D1_voters_df.loc[i, 'Rank2 raw'] = "skipped"
    elif rank2_votes > 1:
        D1_voters_df.loc[i, 'Rank2 raw'] = "overvote"
    elif rank2_votes == 1:
        this_vote_col = [col for col in rank2_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank2 raw'] = this_vote

    rank3_votes = D1_voters_df.loc[i, rank3_cols].sum()
    if rank3_votes == 0:
        D1_voters_df.loc[i, 'Rank3 raw'] = "skipped"
    elif rank3_votes > 1:
        D1_voters_df.loc[i, 'Rank3 raw'] = "overvote"
    elif rank3_votes == 1:
        this_vote_col = [col for col in rank3_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank3 raw'] = this_vote

    rank4_votes = D1_voters_df.loc[i, rank4_cols].sum()
    if rank4_votes == 0:
        D1_voters_df.loc[i, 'Rank4 raw'] = "skipped"
    elif rank4_votes > 1:
        D1_voters_df.loc[i, 'Rank4 raw'] = "overvote"
    elif rank4_votes == 1:
        this_vote_col = [col for col in rank4_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank4 raw'] = this_vote

    rank5_votes = D1_voters_df.loc[i, rank5_cols].sum()
    if rank5_votes == 0:
        D1_voters_df.loc[i, 'Rank5 raw'] = "skipped"
    elif rank5_votes > 1:
        D1_voters_df.loc[i, 'Rank5 raw'] = "overvote"
    elif rank5_votes == 1:
        this_vote_col = [col for col in rank5_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank5 raw'] = this_vote

    rank6_votes = D1_voters_df.loc[i, rank6_cols].sum()
    if rank6_votes == 0:
        D1_voters_df.loc[i, 'Rank6 raw'] = "skipped"
    elif rank6_votes > 1:
        D1_voters_df.loc[i, 'Rank6 raw'] = "overvote"
    elif rank6_votes == 1:
        this_vote_col = [col for col in rank6_cols if D1_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D1_candidates if name in this_vote_col][0]
        D1_voters_df.loc[i, 'Rank6 raw'] = this_vote


100%|████████████████████████████████████| 43669/43669 [01:18<00:00, 554.87it/s]


### Next, clean the votes for VoteKit according to Portland's rules: overvotes and undervotes are both skipped and ballots are transferred to the next ranking.  If the same candidate receives multiple rankings, only keep the highest ranking.
### It appears that the categories 'Write-in-120', 'Write-in-121', and 'Write-in-122' are ALSO skipped and ballots are transferred to the next ranking.  (Equivalently, these "candidates" were eliminated first in a "Round 0" that occurred before the published tabulation rounds.)

In [10]:
D1_voters_df.insert(16, "Rank1 condensed", None)
D1_voters_df.insert(17, "Rank2 condensed", None)
D1_voters_df.insert(18, "Rank3 condensed", None)
D1_voters_df.insert(19, "Rank4 condensed", None)
D1_voters_df.insert(20, "Rank5 condensed", None)
D1_voters_df.insert(21, "Rank6 condensed", None)


In [11]:
D1_candidates_R1 = D1_candidates[:-3]
D1_candidates_R1

['Candace Avalos',
 'Jamie Dunphy',
 'Loretta Smith',
 'Noah Ernst',
 'Terrence Hayes',
 'Steph Routh',
 'Timur Ender',
 'Doug Clove',
 'Peggy Sue Owens',
 'David Linn',
 'Joe Allen',
 'Michael (Mike) Sands',
 'Deian Salazar',
 'Cayle Tern',
 'Thomas Shervey',
 'Joe Furi',
 'Uncertified Write In']

In [12]:
condensed_ballots_R1 = []

for i in tqdm(D1_voters_df.index):
    this_ballot_raw = D1_voters_df.loc[i, ['Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw', 'Rank5 raw', 'Rank6 raw']]
    
    this_ballot_cleaned = [name for name in this_ballot_raw if name in D1_candidates_R1]

    this_ballot_condensed = []
    for name in this_ballot_cleaned:
        if name not in this_ballot_condensed:
            this_ballot_condensed.append(name)    

    if len(this_ballot_condensed) > 0:
        condensed_ballots_R1.append(this_ballot_condensed)
    
    if len(this_ballot_condensed) < 6:
        fill_len = 6 - len(this_ballot_condensed)
        this_ballot_condensed = this_ballot_condensed + ["skipped"]*fill_len

    D1_voters_df.loc[i, "Rank1 condensed"] = this_ballot_condensed[0]
    D1_voters_df.loc[i, "Rank2 condensed"] = this_ballot_condensed[1]
    D1_voters_df.loc[i, "Rank3 condensed"] = this_ballot_condensed[2]
    D1_voters_df.loc[i, "Rank4 condensed"] = this_ballot_condensed[3]
    D1_voters_df.loc[i, "Rank5 condensed"] = this_ballot_condensed[4]
    D1_voters_df.loc[i, "Rank6 condensed"] = this_ballot_condensed[5]
    

100%|███████████████████████████████████| 43669/43669 [00:19<00:00, 2277.25it/s]


### Create two datasets to save: one with the raw data and one with the condensed data.  Keep all the intro columns for the raw data, but not for the cleaned data.

In [13]:
D1_voters_df.columns[0:22]

Index(['RowNumber', 'BoxID', 'BoxPosition', 'BallotID', 'PrecinctID',
       'BallotStyleID', 'PrecinctStyleName', 'ScanComputerName', 'Status',
       'Remade', 'Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw',
       'Rank5 raw', 'Rank6 raw', 'Rank1 condensed', 'Rank2 condensed',
       'Rank3 condensed', 'Rank4 condensed', 'Rank5 condensed',
       'Rank6 condensed'],
      dtype='object')

In [14]:
D1_raw_df = D1_voters_df[D1_voters_df.columns[0:16]]

In [15]:
D1_condensed_df = D1_voters_df[['PrecinctStyleName'] + list(D1_voters_df.columns)[16:22]]

### The raw file is good to go; just rename the ranking columns.

In [16]:
D1_raw_df = D1_raw_df.rename(columns = {
    'Rank1 raw' : 'rank1',
    'Rank2 raw' : 'rank2',
    'Rank3 raw' : 'rank3',
    'Rank4 raw' : 'rank4',
    'Rank5 raw' : 'rank5',
    'Rank6 raw' : 'rank6'
})

In [17]:
D1_raw_df.to_csv("./Cleaned CVRs/Portland_D1_Cleaned.csv", index = False)

### For the condensed file, remove voters who no longer have any votes. (These are the ones who only voted for 'Write-in-120', 'Write-in-121', and/or 'Write-in-122'.)  

In [18]:
D1_condensed_df = D1_condensed_df[D1_condensed_df['Rank1 condensed'] != 'skipped'].reset_index(drop="True")

### Rename the columns:

In [19]:
D1_condensed_df = D1_condensed_df.rename(columns = {
    'PrecinctStyleName': 'Precinct',
    'Rank1 condensed' : 'rank1',
    'Rank2 condensed' : 'rank2',
    'Rank3 condensed' : 'rank3',
    'Rank4 condensed' : 'rank4',
    'Rank5 condensed' : 'rank5',
    'Rank6 condensed' : 'rank6'
})

### Change the format of the entries in the Precinct column to match the precinct shapefile:

In [20]:
set(D1_condensed_df['Precinct'])

{'4406-2',
 '4407-1',
 '4508-1',
 '4509-1',
 '4605-2',
 '4606-2',
 '4607-1',
 '4702-1',
 '4703-1',
 '4704-1',
 '4705-1',
 '4706-1',
 '4707-1',
 '4708-1',
 '4709-1',
 '4710-1',
 '4801-1',
 '4802-1',
 '4803-1',
 '4804-1',
 '4805-1',
 '4806-1',
 '4809-1',
 '4910-1',
 '5001-1',
 '5003-1',
 'CLACK-113-1',
 'CLACK-114-1'}

In [21]:
for i in D1_condensed_df.index:
    if D1_condensed_df.loc[i, 'Precinct'].startswith('CLACK'):
        D1_condensed_df.loc[i, 'Precinct'] = 'C'+ D1_condensed_df.loc[i, 'Precinct'][-5:-2]
    elif D1_condensed_df.loc[i, 'Precinct'].startswith('WASH'):
        D1_condensed_df.loc[i, 'Precinct'] = 'W'+ D1_condensed_df.loc[i, 'Precinct'][-5:-2]
    else:
        D1_condensed_df.loc[i, 'Precinct'] = 'M'+ D1_condensed_df.loc[i, 'Precinct'][0:4]

In [22]:
D1_condensed_df

,Precinct,rank1,rank2,rank3,rank4,rank5,rank6
0,M4703,Terrence Hayes,Loretta Smith,Noah Ernst,skipped,skipped,skipped
1,M4508,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
2,M4508,Loretta Smith,Steph Routh,Timur Ender,David Linn,Candace Avalos,Jamie Dunphy
3,M4802,Michael (Mike) Sands,Doug Clove,Joe Furi,Timur Ender,Deian Salazar,Loretta Smith
4,M4707,Timur Ender,Candace Avalos,Cayle Tern,Steph Routh,Michael (Mike) Sands,Jamie Dunphy
...,...,...,...,...,...,...,...
42866,M4509,Loretta Smith,Candace Avalos,Jamie Dunphy,Timur Ender,skipped,skipped
42867,M4705,Loretta Smith,skipped,skipped,skipped,skipped,skipped
42868,M4708,Steph Routh,Jamie Dunphy,skipped,skipped,skipped,skipped
42869,M4806,Steph Routh,Loretta Smith,Candace Avalos,skipped,skipped,skipped


### Save it:

In [23]:
D1_condensed_df.to_csv("./Cleaned CVRs/Portland_D1_Condensed.csv", index = False)